In [92]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
data_merge_df = pd.merge(mouse_metadata, study_results, how='outer', on="Mouse ID")

# Display the data table for preview
data_merge_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [93]:
## Checking the number of unique timepoints for each mouse ID
group_df = data_merge_df.groupby("Mouse ID")
group_df["Timepoint"].value_counts()

Mouse ID  Timepoint
a203      0            1
          5            1
          10           1
          15           1
          20           1
                      ..
z969      25           1
          30           1
          35           1
          40           1
          45           1
Name: Timepoint, Length: 1888, dtype: int64

In [94]:
## Since we have 1893 rows but only 1888 cummulative unique timestamps per mouse, there are a few timepoint errors (i.e. timepoints with multiple values assigned for a certian mouse ID)
group = data_merge_df.groupby('Mouse ID')

## If we have a value above 1, then this is indeed the case
group["Timepoint"].value_counts().max()

2

In [95]:
## We fix this by grouping by Mouse ID and the Timepoints and transforming the df by dropping ID's with sizes above 1 
clean_df = data_merge_df[data_merge_df.groupby("Mouse ID")['Timepoint'].transform('size') > 1]
clean_df

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [100]:
## Now our # of rows and # of unique timepoint values per mouse match

## Now let's check for NA valued entries
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1881 entries, 0 to 1892
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Mouse ID            1881 non-null   object 
 1   Drug Regimen        1881 non-null   object 
 2   Sex                 1881 non-null   object 
 3   Age_months          1881 non-null   int64  
 4   Weight (g)          1881 non-null   int64  
 5   Timepoint           1881 non-null   int64  
 6   Tumor Volume (mm3)  1881 non-null   float64
 7   Metastatic Sites    1881 non-null   int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 132.3+ KB


In [101]:
## Our dataframe is free of null entries so we are now good to go for analysis!

In [102]:
## SUMMARY STATISTICS

##Calculate mean, median, variance, standard deviation, and SEM of the tumor volume by drug regimen
regimen_df = clean_df.groupby("Drug Regimen")
tumor_volume_mean = regimen_df["Tumor Volume (mm3)"].mean()
tumor_volume_med = regimen_df["Tumor Volume (mm3)"].median()
tumor_volume_var = regimen_df["Tumor Volume (mm3)"].var()
tumor_volume_std = regimen_df["Tumor Volume (mm3)"].std()
tumor_volume_sem = regimen_df["Tumor Volume (mm3)"].sem()

## Shove into a dataframe
summary_stat_df = pd.DataFrame({"Tumor Volume Mean" : tumor_volume_mean, 
                                "Tumor Volume Median" : tumor_volume_med, 
                                "Tumor Volume Variance" : tumor_volume_var, 
                                "Tumor Volume Standard Dev" : tumor_volume_std, 
                                "Tumor Volume SEM" : tumor_volume_sem
                               }).round(3)
summary_stat_df

,Tumor Volume Mean,Tumor Volume Median,Tumor Volume Variance,Tumor Volume Standard Dev,Tumor Volume SEM
Drug Regimen,,,,,
Capomulin,40.676,41.558,24.948,4.995,0.329
Ceftamin,52.766,51.910,38.836,6.232,0.472
Infubinol,52.885,51.821,43.129,6.567,0.492
Ketapril,55.346,53.740,68.150,8.255,0.605
Naftisol,54.433,52.545,65.935,8.120,0.599
Placebo,54.084,52.316,61.051,7.814,0.582
Propriva,52.368,50.910,42.278,6.502,0.514
Ramicane,40.217,40.673,23.487,4.846,0.321
Stelasyn,54.284,52.488,59.304,7.701,0.574


In [99]:
## BAR AND PIE CHARTS

